In [2]:
# Imports
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import datetime
import time

# Override yfinance API
yf.pdr_override()
from pathlib import Path
import numpy as np

In [3]:
# Get tickers for all S&P 500 stocks and replace "." with "-" for compatibility with Yahoo Finance
sp500_tickers = si.tickers_sp500()
sp500_tickers = [ticker.replace(".", "-") for ticker in sp500_tickers]

In [4]:
# scan a list of stocks from a csv file of Investors Business Daily recommendations
csv_path = Path("FFTY_IBD50.csv")
ibd_df = pd.read_csv(csv_path)

In [5]:
# Define S&P 500 index
#sp500_index = 'SPY'
ibd_index = 'FFTY'
# Define date range for stock data
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

In [6]:
# Create empty list to store relative returns for each stock
relative_returns = []

In [7]:
ibd = pdr.get_data_yahoo(ibd_index, start_date, end_date)
ibd

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-27,23.139999,23.370001,23.139999,23.240000,23.095100,41000
2023-02-28,23.270000,23.410000,23.190001,23.190001,23.045412,26400
2023-03-01,23.209999,23.389999,23.129999,23.275000,23.129881,14400
2023-03-02,22.900000,23.305000,22.840000,23.290001,23.144789,15500
2023-03-03,23.520000,23.850000,23.400000,23.809999,23.661545,56400
...,...,...,...,...,...,...
2024-02-16,27.190001,27.379999,26.889999,26.990000,26.990000,106300
2024-02-20,26.850000,26.850000,25.820000,26.170000,26.170000,353900
2024-02-21,25.620001,25.690001,25.363001,25.580000,25.580000,136900


In [8]:
# Retrieve historical price data for the S&P 500 index
#sp500_df = pdr.get_data_yahoo(sp500_index, start_date, end_date)
ibd = pdr.get_data_yahoo(ibd_index, start_date, end_date)
#sp500_df['Percent Change'] = sp500_df['Adj Close'].pct_change()
ibd['Percent Change'] = ibd['Adj Close'].pct_change() * 100 
#sp500_returns = sp500_df['Percent Change'].cumprod()
ibd_returns = ibd['Percent Change'].cumprod()
#sp500_return = sp500_returns.iloc[-1]
ibd_return = ibd_returns.iloc[-1]

[*********************100%%**********************]  1 of 1 completed


In [9]:
ibd.tail(50)

,Open,High,Low,Close,Adj Close,Volume,Percent Change
Date,,,,,,,
2023-12-12,23.690001,24.020000,23.580000,23.940001,23.790735,55600,1.012653
2023-12-13,24.010000,24.459999,23.900999,24.440001,24.287619,115800,2.088558
2023-12-14,24.639999,24.820000,24.190001,24.410000,24.257805,111700,-0.122753
2023-12-15,24.450001,24.620001,24.280001,24.420000,24.267742,144700,0.040965
2023-12-18,24.469999,24.719999,24.430000,24.639999,24.486370,133200,0.900899
2023-12-19,24.740000,24.950001,24.719999,24.889999,24.734812,142600,1.014612
2023-12-20,24.750000,24.919001,24.280001,24.280001,24.128616,263300,-2.450779
2023-12-21,24.610001,24.749001,24.490000,24.740000,24.585747,172200,1.894557
2023-12-22,24.730000,24.830000,24.590000,24.709999,24.555933,151700,-0.121264


In [73]:
ibd_test = ibd['Percent Change'] == 0

In [75]:
ibd_test.unique()

array([False])

In [57]:
ibd_return

0.0

In [58]:
#sp500_returns
ibd_returns

In [63]:
df = pd.DataFrame([[2.0, 1.0],
                   [0.0, np.nan],
                   [1.0, 0.0],
                    [3.0, 2.0]],
                  columns=list('AB'))

In [64]:
df.cumprod()

,A,B
0,2.0,1.0
1,0.0,NaN
2,0.0,0.0
3,0.0,0.0


In [12]:
# Iterate over all S&P 500 stocks to calculate their relative returns
for ticker in sp500_tickers:
    # Download historical data as CSV for each stock to speed up the process
    stock_df = pdr.get_data_yahoo(ticker, start_date, end_date)
    stock_df.to_csv(f'{ticker}.csv')
    
    # Calculate percent change column
    stock_df['Percent Change'] = stock_df['Adj Close'].pct_change()
    
    # Calculate the relative return with double weight for the most recent quarter
    stock_returns = stock_df['Percent Change'].cumprod()
    stock_return = (stock_returns.iloc[-10] * 2 + stock_returns.iloc[-63]) / 3 # Double weight for the most recent quarter
    relative_return = stock_return / sp500_return
    relative_returns.append(relative_return)
    
    # Print relative return for each stock
    print(f'Ticker: {ticker}; Relative Return against S&P 500: {relative_return}\n')
    
    # Pause for 1 second to avoid overloading the server with requests
    time.sleep(1)

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['-']: Exception('%ticker%: No timezone found, symbol may be delisted')


IndexError: single positional indexer is out-of-bounds

In [48]:
# Iterate over all S&P 500 stocks to calculate their relative returns
for ticker in ibd_df:
    # Download historical data as CSV for each stock to speed up the process
    stock_df = pdr.get_data_yahoo(ticker, start_date, end_date)
    stock_df.to_csv(f'{ticker}.csv')
    
    # Calculate percent change column
    stock_df['Percent Change'] = stock_df['Adj Close'].pct_change()
    
    # Calculate the relative return with double weight for the most recent quarter
    stock_returns = stock_df['Percent Change'].cumprod()
    stock_return = (stock_returns.iloc[-10] * 2 + stock_returns.iloc[-50]) / 3 # Double weight for the most recent quarter
    relative_return = stock_return / ibd_return
    relative_returns.append(relative_return)
    
    # Print relative return for each stock
    print(f'Ticker: {ticker}; Relative Return against S&P 500: {relative_return}\n')
    
    # Pause for 1 second 
    #to avoid overloading the server with requests
    time.sleep(1)
    

[*********************100%%**********************]  1 of 1 completed
Ticker: FFTY; Relative Return against S&P 500: 0.0



In [45]:
print(stock_return)

In [46]:
print(stock_returns)

Date
2023-02-21             NaN
2023-02-22    3.948955e-03
2023-02-23    4.660072e-05
2023-02-24   -4.025948e-07
2023-02-27   -5.087237e-09
                  ...     
2024-02-12    0.000000e+00
2024-02-13   -0.000000e+00
2024-02-14   -0.000000e+00
2024-02-15   -0.000000e+00
2024-02-16    0.000000e+00
Name: Percent Change, Length: 250, dtype: float64


In [20]:
# Create dataframe with relative returns and corresponding RS ratings
#rs_df = pd.DataFrame(list(zip(sp500_tickers, relative_returns)), columns=['Ticker', 'Relative Return'])


In [22]:
rs_df['RS_Rating'] = rs_df['Relative Return'].rank(pct=True) * 100

In [24]:
rs_df

,Ticker,Relative Return,RS_Rating
0,A,NaN,NaN
1,AAL,NaN,NaN
2,AAP,NaN,NaN
3,AAPL,NaN,NaN
4,ABBV,NaN,NaN
...,...,...,...
149,DXC,1.222658,76.056338
150,DXCM,0.370674,52.816901
151,EA,1.780410,86.619718
152,EBAY,0.547714,59.859155


In [23]:
# Print RS Ratings for all stocks
print(rs_df)

    Ticker  Relative Return  RS_Rating
0        A              NaN        NaN
1      AAL              NaN        NaN
2      AAP              NaN        NaN
3     AAPL              NaN        NaN
4     ABBV              NaN        NaN
..     ...              ...        ...
149    DXC         1.222658  76.056338
150   DXCM         0.370674  52.816901
151     EA         1.780410  86.619718
152   EBAY         0.547714  59.859155
153    ECL        -0.625660   7.042254

[154 rows x 3 columns]
